In [1]:
import pandas as pd
import pymssql

In [3]:
server="mssql.ti.local:1435"
user="bhdmq"
password="************************"
dbname="BullhornDM"

conn = pymssql.connect(server, user, password, dbname, charset="ISO-8859-1")
cursor = conn.cursor()

In [4]:
cursor.execute("""
SELECT 
       CU.customText1				[EMPLOYEE_ID]
	  ,isnull(case when len(UPPER(CC.[customText1])) <5 then '0' else UPPER(CC.[customText1]) end,'0') [CLIENT_ID]
	  ,UPPER([action]) as			[ACTION]
      ,CONVERT(varchar,(note.[dateAdded]),101) as				[ACTION_DATE]
	  ,UPPER([trackTitle])			[TYPE]
	 ,count(distinct note.noteid)							[COUNT]
	 --,1 [COUNT]

  FROM [BULLHORNDM].[dbo].[Note] Note
  Left JOIN [BULLHORNDM].[dbo].[NoteEntity] NE on Note.noteID = NE.noteID
  Left JOIN [BULLHORNDM].[dbo].[CorporateUser] CU on Note.[commentingPersonID] = CU.[userID]
  Left JOIN [BULLHORNDM].[dbo].[ClientCorporation]CC on note.personReferenceID = cc.clientCorporationID
       
  where len(CU.customText1) >1
 

  group by

        CU.customText1				
	  ,case when len(UPPER(CC.[customText1])) <5 then '0' else 	UPPER(CC.[customText1]) end	
      ,UPPER([action]) 
      ,CONVERT(varchar,(note.[dateAdded]),101) 
	  ,UPPER([trackTitle])	
               
""")

In [5]:
rows = cursor.fetchall()



df1 = pd.DataFrame(rows)
df1.columns = [item[0] for item in cursor.description]
df1.columns = ['EMPLOYEE_ID','CLIENT_ID','ACTION', '445_DATE','CONTACT_TYPE', 'COUNT']
df1['445_DATE'] = pd.to_datetime(df1['445_DATE'])



In [6]:
df1.head()

,EMPLOYEE_ID,CLIENT_ID,ACTION,445_DATE,CONTACT_TYPE,COUNT
0,10035,0,APPOINTMENT,2019-04-02,None,1
1,10035,0,APPOINTMENT,2019-05-23,None,1
2,10035,0,APPOINTMENT,2019-06-05,None,1
3,10035,0,APPOINTMENT,2019-06-05,PROSPECT,1
4,10035,0,EMAIL,2019-04-02,PROSPECT,1


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190448 entries, 0 to 190447
Data columns (total 6 columns):
EMPLOYEE_ID     190448 non-null object
CLIENT_ID       190448 non-null object
ACTION          190448 non-null object
445_DATE        190448 non-null datetime64[ns]
CONTACT_TYPE    81297 non-null object
COUNT           190448 non-null int64
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 8.7+ MB
